In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from tensorflow.keras.preprocessing import image

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data_dir_training = "/Users/aaryangulia/Downloads/archive-2/Train_Alphabet"
data_dir_testing = "/Users/aaryangulia/Downloads/archive-2/Test_Alphabet"

In [3]:
# Define the number of classes
num_classes = len(os.listdir(data_dir_training)) - 1

# Define the image size and batch size
img_size = (224, 224)
batch_size = 32

# Use ImageDataGenerator for data loading and augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    validation_split=0.2,  # Split data into training and validation sets
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20% of the height
    shear_range=0.2,  # Shear intensity (shear angle in radians)
    zoom_range=0.2,  # Randomly zoom images by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

In [4]:
train_generator = datagen.flow_from_directory(
    data_dir_training,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
    subset='training'  # Specify this as training data
)

# Create validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir_training,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify this as validation data
)

Found 19440 images belonging to 27 classes.
Found 4860 images belonging to 27 classes.


In [7]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

In [19]:
base_model = tf.keras.models.load_model('models/epoch3_model')

# Freeze the layers of the base model
base_model.trainable = False

# Add a new output layer for 27 classes
new_output = Dense(27, activation='softmax')

# Connect the base model and the new output layer
model_output = new_output(base_model.output)

# Create the new model
new_model = tf.keras.Model(inputs=base_model.input, outputs=model_output)

# Compile the new model and train it on the new dataset
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
checkpoint_callback = ModelCheckpoint(filepath='tranferlearn_original_model.h5', 
                                      save_best_only=False, 
                                      verbose=1)
#training model
print("\n Model summary: ")
print(new_model.summary())
print("\n Model Training: ")
history = new_model.fit(
    train_generator,
    epochs=5,  # You can adjust the number of epochs based on your training needs
    validation_data=validation_generator,
    callbacks=[checkpoint_callback]
)


 Model summary: 
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224_input  [(None, 224, 224, 3)]     0         
  (InputLayer)                                                   
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 29)                37149     
                                                                 
 dense_3 (Dense)             (None, 27)                810       
                                         

KeyboardInterrupt: 